In [1]:
""" First tests with WRN - 16-4. """
import keras
import numpy as np
import keras_wrn as wrn

import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras import regularizers, optimizers
import keras.callbacks as callbacks

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

c:\users\eric\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#data augmentation
datagen = ImageDataGenerator(rotation_range=0,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             horizontal_flip=True,
                             fill_mode="reflect")
datagen.fit(x_train)

In [3]:
def schedule(x):
    if x < 60:
        return 0.1
    elif x < 120:
        return 0.1*0.2
    elif x < 160:
        return 0.1*0.2*0.2
    else:
        return 0.1*0.2*0.2*0.2

In [ ]:
def run(n,k,act,dropout):
    init_shape = (3, 32, 32) if K.image_dim_ordering() == 'th' else (32, 32, 3)
    # For WRN-16-8 put N = 16, k = 8
    # For WRN-28-10 put N = 28, k = 10
    # For WRN-40-4 put N = 40, k = 4
    model = wrn.build_model(init_shape, num_classes, n, k, act=act, dropout=dropout)

    print("Model Created")
    batch_size  = 128
    epochs = 200

    opt = keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=True)
    lr_1 = keras.callbacks.LearningRateScheduler(schedule)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    print("Finished compiling")

    ####################
    # Network training #
    ####################

    print("Gonna fit the model")
    his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
    print(his.history)
    return his.history

In [ ]:
# Creating custom activation: E-swish. For ReLU just type "relu"
def e_swish(x):
    return 1.375*x*K.sigmoid(x)

# WRN-16-4 - no dropout w/ custom Activation
run(n=16, k=4, act=e_swish, dropout=0) # Replace e_swish by "relu"